<a href="https://colab.research.google.com/github/pollinations/pollinations/blob/alternative-pollinate/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pollinator

Does the heavy-lifting for **Pollinations.AI** on Google Colab's free GPUs.

Click *Runtime - Run All* and switch back to the **Pollinations** tab of the browser.

![](https://pollinations.ai/Runtime-RunAll.png?)

In [ ]:
#@title Config and Requirements
!echo "adding nameserver 8.8.8.8 to /etc/resolv.conf"
!echo -e "nameserver 8.8.8.8\n$(cat /etc/resolv.conf)" > /etc/resolv.conf

!pip install --upgrade pip
!pip install coolname gdown papermill json5
from IPython.display import clear_output
from coolname import generate_slug
from os.path import join, basename
from shlex import quote

from glob import glob
from json5 import loads

from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

clear_output()
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
ipfs_root = "/content/ipfs"
worker_root = "/content"
notebook_path = join(ipfs_root,"input", "notebook.ipynb")

print(f"---Settings---\nIPFS_ROOT: {ipfs_root}")

adding nameserver 8.8.8.8 to /etc/resolv.conf
     |████████████████████████████████| 1.6 MB 5.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None


In [ ]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!mkdir -p $worker_root


!echo "Created IPFS resource" > $ipfs_root/output/log
!echo -n '"initializing"' > $ipfs_root/output/status

clear_output()

In [ ]:
#@title Install Pollinations CLI

%cd /content
!git clone https://github.com/voodoohop/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout alternative-pollinate 
!git pull
!npm run install_backend
%cd -

clear_output()

In [ ]:
#@title Install and Run IPFS daemon &

!/content/pollinations/ipfs/ipfsd_install_colab.sh
!nohup  /content/pollinations/ipfs/ipfsd_forever.sh &
!sleep 5
clear_output()


node_id=!ipfs id -f="<id>\n"
node_id=node_id[0]

print("---Node ID---:", node_id)

In [ ]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `https://pollinations.ai/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe)
  const iframe2 = document.createElement("iframe")
  iframe2.src = `http://localhost:3000/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe2)
'''))




In [ ]:
#@title Receive first IPFS state. Watch and send ipfs output folder &

def get_params():
  parameters={}
  for path in glob(f"{ipfs_root}/input/*"):
    key = basename(path)
    if key == "notebook.ipynb":
      continue
    with open(path,'r') as f: 
      value = f.read()
      #print("reading parameter", value)
      try:
        parameters[key] = loads(value)
      except:
        parameters[key] = ""
  return parameters

def get_form_action():
  params = get_params()
  return "formAction" in params and params["formAction"]
  

def wait_submitted(skip_pollinate=False):
  print("Waiting for form submission")
  while not get_form_action() == "submit":
    if skip_pollinate:
      !sleep 2
    else:
      !DEBUG=* pollinate -p $ipfs_root/input --ipns --receive --once > /content/cid

def set_status(status):
  status = "'" + status + "'"
  !echo -n  "$status" > $ipfs_root/output/status
  print("Set status to",status)

def post_content():
  cid_all=!tail -1 /content/cid
  cid = cid_all[0]
  social_post_url = "https://pollinations.ai/.netlify/functions/social-post/"+cid
  !curl $social_post_url > $ipfs_root/output/log

wait_submitted()

print("Preparing notebook at",notebook_path,"for Papermill execution")

!python /content/pollinations/pollinations/prepare_for_papermill.py $notebook_path

#!nohup pollinate -p $ipfs_root --ipns --debounce 1000 --send > /content/cid &

clear_output()

In [ ]:
#@title Notebook Runner
from time import sleep
import sys
import matplotlib
matplotlib.use('Agg')
output_path = ipfs_root+"/output"

first_run=True


def run(parameters):
  print("Running",parameters)
 
  parameters["output_path"] = output_path
  _params = [["-p", quote(key), quote(str(value))] for key, value in parameters.items()]
  params = [str(val) for sublist in _params for val in sublist]
  notebook_out_path = f"/content/notebook_output.ipynb"
  cmd = ["papermill", notebook_path, notebook_out_path] + params + ["--log-output"]
  cmd = " ".join(cmd)
  
  print("Running papermill cmd",cmd)
  !rm -rv $ipfs_root/output/*
  !echo "$cmd" > /content/run.sh

  !echo "Starting..." > $ipfs_root/output/log
  set_status("running")
  !pollinate --execute "/bin/bash /content/run.sh"  --ipns --execute "bash run.sh" -l $ipfs_root/output/log -p $ipfs_root --ipns >> /content/cid
 
  post_content()

      
  
while True:
  set_status("waiting")
  if not first_run:
    #!rm -v $output_path/*    
    wait_submitted()
    !python /content/pollinations/pollinations/prepare_for_papermill.py $notebook_path
  first_run = False
  parameters = get_params()
  print("New parameters", parameters)
  #!rm -v $output_path/*
  #!rm -v $ipfs_root/input/formAction
  run(parameters)
  !rm -v $ipfs_root/input/formAction
  !sleep 3
